In [1]:
import os
os.chdir("../../")#与easymlops同级目录

### 拆分数据

In [2]:
import pandas as pd
data=pd.read_csv("./data/demo.csv")
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
x_train=data[:500]
x_test=data[500:]
y_train=x_train["Survived"]
y_test=x_test["Survived"]
del x_train["Survived"]
del x_test["Survived"]

### pipeline运行到指定层
我们有时候可能想看看pipeline过程中特征逐层的变化情况，以及复用别人的特征工程（但又不需要最后几步的变化），transform/transform_single中的run_to_layer就可以排上用场了

In [4]:
from easymlops import TablePipeLine
from easymlops.table.preprocessing import *
from easymlops.table.encoding import *
from easymlops.table.perfopt import *
from easymlops.table.decomposition import *
from easymlops.table.ensemble import *

In [5]:
table=TablePipeLine()
table.pipe(FixInput())\
  .pipe(FillNa())\
  .pipe(Parallel([OneHotEncoding(cols=["Pclass","Sex"],drop_col=False),
                 TablePipeLine().pipe(TargetEncoding(cols=["Embarked","Cabin"],y=y_train,name="target_encoding")).pipe(FillNa()),
                 WOEEncoding(cols=["Sex","Pclass"],y=y_train),
                 TablePipeLine().pipe(LabelEncoding(cols=["Name","Ticket"])).pipe(DropCols(cols=["Name","Sex","Ticket","Cabin","Emarked"]))]))\
  .pipe(TargetEncoding(cols=["Name","Cabin","Embarked","Ticket"],y=y_train))\
  .pipe(FillNa())\
  .pipe(Normalizer())\
  .pipe(PCADecomposition(n_components=8))

x_test_new=table.fit(x_train).transform(x_test)
x_test_new.head(5)

,0,1,2,3,4,5,6,7
500,-2.310728,-0.090716,-0.192874,0.510718,0.203634,-1.868558,-0.259263,-0.254359
501,0.178031,-1.920634,-0.317246,1.035497,-1.908379,-1.607179,-1.486184,0.609996
502,0.113005,-2.265333,-0.104388,1.140618,-1.467519,-1.499789,-2.154350,-0.043210
503,-0.023234,-1.645083,-0.733197,-0.026390,-2.074293,-2.249345,0.102076,-0.128000
504,2.473881,0.879680,0.042627,-0.233503,-1.788442,-1.923288,-1.016605,-1.354828


浅层访问,run_to_layer设置为指定层的index

In [6]:
#查看第1层的数据
table.transform(x_test,run_to_layer=1).head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
500,501,3,"Calic, Mr. Petar",male,17.0,0,0,315086,8.664062,nan,S
501,502,3,"Canavan, Miss. Mary",female,21.0,0,0,364846,7.750000,nan,Q
502,503,3,"O'Sullivan, Miss. Bridget Mary",female,0.0,0,0,330909,7.628906,nan,Q
503,504,3,"Laitinen, Miss. Kristina Sofia",female,37.0,0,0,4135,9.585938,nan,S
504,505,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.500000,B79,S


In [7]:
#查看第2层的数据
table.transform(x_test,run_to_layer=2).head(5)

,Pclass_3,Pclass_1,Pclass_2,Sex_male,Sex_female,Name,Sex,Ticket,Cabin,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked
500,1,0,0,1,0,"Calic, Mr. Petar",1.111379,315086,nan,501,3,17.0,0,0,8.664062,S
501,1,0,0,0,1,"Canavan, Miss. Mary",-1.569990,364846,nan,502,3,21.0,0,0,7.750000,Q
502,1,0,0,0,1,"O'Sullivan, Miss. Bridget Mary",-1.569990,330909,nan,503,3,0.0,0,0,7.628906,Q
503,1,0,0,0,1,"Laitinen, Miss. Kristina Sofia",-1.569990,4135,nan,504,3,37.0,0,0,9.585938,S
504,0,1,0,0,1,"Maioni, Miss. Roberta",-1.569990,110152,B79,505,1,16.0,0,0,86.500000,S


In [8]:
#查看倒数第3层的数据
table.transform(x_test,run_to_layer=-3).head(5)

,Pclass_3,Pclass_1,Pclass_2,Sex_male,Sex_female,Name,Sex,Ticket,Cabin,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked
500,1,0,0,1,0,0.0,1.111379,0.0,0.317829,501,3,17.0,0,0,8.664062,0.334254
501,1,0,0,0,1,0.0,-1.569990,0.0,0.317829,502,3,21.0,0,0,7.750000,0.511111
502,1,0,0,0,1,0.0,-1.569990,0.0,0.317829,503,3,0.0,0,0,7.628906,0.511111
503,1,0,0,0,1,0.0,-1.569990,0.0,0.317829,504,3,37.0,0,0,9.585938,0.334254
504,0,1,0,0,1,0.0,-1.569990,1.0,0.000000,505,1,16.0,0,0,86.500000,0.334254


嵌套层的范围:将run_to_layer设置为list结构，分别指定逐层的index

In [9]:
#查看第2层(Parallel)的第0层(OneHotEncoding)
table.transform(x_test,run_to_layer=[2,0]).head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_3,Pclass_1,Pclass_2,Sex_male,Sex_female
500,501,3,"Calic, Mr. Petar",male,17.0,0,0,315086,8.664062,nan,S,1,0,0,1,0
501,502,3,"Canavan, Miss. Mary",female,21.0,0,0,364846,7.750000,nan,Q,1,0,0,0,1
502,503,3,"O'Sullivan, Miss. Bridget Mary",female,0.0,0,0,330909,7.628906,nan,Q,1,0,0,0,1
503,504,3,"Laitinen, Miss. Kristina Sofia",female,37.0,0,0,4135,9.585938,nan,S,1,0,0,0,1
504,505,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.500000,B79,S,0,1,0,0,1


In [10]:
#运行到第2层(Parallel)的第1层(TablePipeLine)的第0层(TargetEncoding)
table.transform(x_test,run_to_layer=[2,1,0]).head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
500,501,3,"Calic, Mr. Petar",male,17.0,0,0,315086,8.664062,0.317829,0.334254
501,502,3,"Canavan, Miss. Mary",female,21.0,0,0,364846,7.750000,0.317829,0.511111
502,503,3,"O'Sullivan, Miss. Bridget Mary",female,0.0,0,0,330909,7.628906,0.317829,0.511111
503,504,3,"Laitinen, Miss. Kristina Sofia",female,37.0,0,0,4135,9.585938,0.317829,0.334254
504,505,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.500000,0.000000,0.334254


命名访问:如果有对模块命名，可以通过指定名称的方式访问

In [11]:
#查看到模块名为target_encoding的数据
table.transform(x_test,run_to_layer="target_encoding").head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
500,501,3,"Calic, Mr. Petar",male,17.0,0,0,315086,8.664062,0.317829,0.334254
501,502,3,"Canavan, Miss. Mary",female,21.0,0,0,364846,7.750000,0.317829,0.511111
502,503,3,"O'Sullivan, Miss. Bridget Mary",female,0.0,0,0,330909,7.628906,0.317829,0.511111
503,504,3,"Laitinen, Miss. Kristina Sofia",female,37.0,0,0,4135,9.585938,0.317829,0.334254
504,505,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.500000,0.000000,0.334254


### 性能、一致性测试

In [12]:
table.auto_test(x_test[:10])


###################################################################
 1.一致性测试和性能测试:check_transform_function                      
###################################################################
(<class 'easymlops.table.preprocessing.core.FixInput'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.preprocessing.onevar_operation.FillNa'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.ensemble.Parallel'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.encoding.TargetEncoding'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.preprocessing.onevar_operation.FillNa'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.preprocessing.onevar_operation.Normalizer'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easym

### 持久化测试

In [13]:
table.save("ml.pkl")

In [14]:
table=TablePipeLine()
table.pipe(FixInput())\
  .pipe(FillNa())\
  .pipe(Parallel([OneHotEncoding(cols=["Pclass","Sex"],drop_col=False),
                 TablePipeLine().pipe(TargetEncoding(cols=["Embarked","Cabin"],y=y_train,name="target_encoding")).pipe(FillNa()),
                 WOEEncoding(cols=["Sex","Pclass"]),
                 TablePipeLine().pipe(LabelEncoding(cols=["Name","Ticket"])).pipe(DropCols(cols=["Name","Sex","Ticket","Cabin","Emarked"]))]))\
  .pipe(TargetEncoding(cols=["Name","Cabin","Embarked","Ticket"]))\
  .pipe(FillNa())\
  .pipe(Normalizer())\
  .pipe(PCADecomposition(n_components=8))


table.load("ml.pkl")

In [15]:
table.auto_test(x_test[:10])


###################################################################
 1.一致性测试和性能测试:check_transform_function                      
###################################################################
(<class 'easymlops.table.preprocessing.core.FixInput'>) module check [transform] complete,speed:[0.29ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.preprocessing.onevar_operation.FillNa'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.ensemble.Parallel'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.encoding.TargetEncoding'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.preprocessing.onevar_operation.FillNa'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.preprocessing.onevar_operation.Normalizer'>) module check [transform] complete,speed:[0.06ms]/it,cpu:[0%],memory:[0K]
(<class 'eas

In [16]:
table.transform(x_test[:5])

,0,1,2,3,4,5,6,7
500,-2.310728,-0.090716,-0.192874,0.510718,0.203634,-1.868558,-0.259263,-0.254359
501,0.178031,-1.920634,-0.317246,1.035497,-1.908379,-1.607179,-1.486184,0.609996
502,0.113005,-2.265333,-0.104388,1.140618,-1.467519,-1.499789,-2.154350,-0.043210
503,-0.023234,-1.645083,-0.733197,-0.026390,-2.074293,-2.249345,0.102076,-0.128000
504,2.473881,0.879680,0.042627,-0.233503,-1.788442,-1.923288,-1.016605,-1.354828


In [17]:
x_test_new.head(5)

,0,1,2,3,4,5,6,7
500,-2.310728,-0.090716,-0.192874,0.510718,0.203634,-1.868558,-0.259263,-0.254359
501,0.178031,-1.920634,-0.317246,1.035497,-1.908379,-1.607179,-1.486184,0.609996
502,0.113005,-2.265333,-0.104388,1.140618,-1.467519,-1.499789,-2.154350,-0.043210
503,-0.023234,-1.645083,-0.733197,-0.026390,-2.074293,-2.249345,0.102076,-0.128000
504,2.473881,0.879680,0.042627,-0.233503,-1.788442,-1.923288,-1.016605,-1.354828
